In [1]:
import pandas as pd

## Pretrained a model:

In [2]:
data_original = pd.read_csv("chv.csv", sep='\t') #avoid error tokenizind data

df = data_original
df = df[[
    "Term",
    "General SNOMED Label",
    "Example"]]

df = df.drop_duplicates()

df.columns = ['Term', 'SNOMED', 'ExampleText']

df[:5]

,Term,SNOMED,ExampleText
0,5FU,Fluorouracil,I can ' t do 5FU because it causes cardiotoxic...
1,5FU,Fluorouracil,He is starting 5FU and Leucovorin and Hercepti...
2,5FU,Fluorouracil,Around treatment 9 they realized that I was al...
3,5HTP,Oxitriptan,"Aside from edibles I take Gaba , Taurine , sub..."
4,5HTP,Oxitriptan,I was recomended the 5HTP and as I said it ini...


In [3]:
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
import nltk


df = df.dropna(subset=['ExampleText'])
df_text = df['ExampleText'].unique()

processed_text = []
count = 0

for text in df_text:
    #print(text)
    #lowecase
    text = text.lower()

    #removing stopwords
    stop_words= stopwords.words('english')
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])

    #removing hastags, mentions, numbers and more
    #text = re.sub(r'\w*\d+\w*', '', text) #remove numbers #with this a1c will be removed
    text = re.sub("@\S+", " ", text) #remove mentions
    text = re.sub("https*\S+", " ", text) #remove links
    text = re.sub("\'\w+", '', text) #remove ticks a next character
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

    #print(text)
    #add the text to the array
    processed_text.append(text)



In [4]:
tokenized_text = [word_tokenize(i) for i in processed_text] #use this as an input for the similarities calculation
flat_list = [item for sublist in tokenized_text for item in sublist]
tokenized_text_words = [nltk.word_tokenize(sent) for sent in flat_list] #use this an an input for word2vec model
#entity_data = pd.DataFrame(processed_text)
print(tokenized_text_words)

[['5fu'], ['causes'], ['cardiotoxicity'], ['reason'], ['need'], ['get'], ['new'], ['treatment'], ['plan'], ['figured'], ['starting'], ['5fu'], ['leucovorin'], ['herceptin'], ['her2'], ['positive'], ['around'], ['treatment'], ['9'], ['realized'], ['allergic'], ['oxaliplatin'], ['continued'], ['5fu'], ['treatment'], ['12'], ['aside'], ['edibles'], ['take'], ['gaba'], ['taurine'], ['sublingual'], ['melatonin'], ['magnesium'], ['5htp'], ['recomended'], ['5htp'], ['said'], ['initially'], ['worked'], ['havn'], ['taking'], ['since'], ['mentioned'], ['amino'], ['acids'], ['l'], ['tryptophan'], ['5htp'], ['precursors'], ['serotonin'], ['200'], ['mg'], ['per'], ['day'], ['blood'], ['work'], ['shows'], ['enough'], ['active'], ['6mp'], ['blood'], ['stream'], ['effective'], ['three'], ['years'], ['ago'], ['remicade'], ['6mp'], ['pred'], ['time'], ['cimzia'], ['failed'], ['go'], ['along'], ['failures'], ['humira'], ['remicade'], ['6mp'], ['mesalamine'], ['oral'], ['enemas'], ['last'], ['several'], [

In [7]:
df_terms = df['Term'].unique()

processed_terms = []

for text in df_terms:
    #print(text)
    #lowecase
    text = text.lower()

    #removing stopwords
    stop_words= stopwords.words('english')
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])

    #removing hastags, mentions, numbers and more
    #text = re.sub(r'\w*\d+\w*', '', text) #remove numbers #with this a1c will be removed
    text = re.sub("@\S+", " ", text) #remove mentions
    text = re.sub("https*\S+", " ", text) #remove links
    text = re.sub("\'\w+", '', text) #remove ticks a next character
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)

    #print(text)
    #add the text to the array
    processed_terms.append(text)



In [8]:
tokenized_terms = [word_tokenize(i) for i in processed_terms] #use this as an input for the similarities calculation
flat_list_terms = [item for sublist in tokenized_terms for item in sublist]
tokenized_term_words = [nltk.word_tokenize(sent) for sent in flat_list_terms] #use this an an input for word2vec model
#entity_data = pd.DataFrame(processed_text)
#print(tokenized_text_words)

In [9]:
print(len(processed_text))
print(len(tokenized_text_words))
print(len(processed_terms))
print(len(tokenized_term_words))

19910
212005
6403
9272


In [10]:
words_for_model = tokenized_text_words + tokenized_term_words #len is 26000

In [11]:
words_for_model[:10]

[['5fu'],
 ['causes'],
 ['cardiotoxicity'],
 ['reason'],
 ['need'],
 ['get'],
 ['new'],
 ['treatment'],
 ['plan'],
 ['figured']]

In [13]:
from gensim.models import Word2Vec

model_for_owl = Word2Vec(words_for_model, min_count=2, workers=3, window =3, sg = 1)
print(model_for_owl)
model_for_owl.save('word2vec_owl_.model')

Word2Vec<vocab=10752, vector_size=100, alpha=0.025>


In [3]:
from owl2vec_star import owl2vec_star
from gensim.models import KeyedVectors

# unable to run this code (computer problem)
#Parameters:
# ontology_file
# config_file
# uri_doc
# lit_doc
# mix_doc
gensim_model = owl2vec_star.extract_owl2vec_model("./case_studies/snomedct/snomedct-201907_module_size_182321.owl", "./default.cfg", True, True, True)

output_folder="./cache/output/"

#Gensim format
gensim_model.save(output_folder+"SMDontology3.embeddings")
#Txt format
gensim_model.wv.save_word2vec_format(output_folder+"SMDontology3.embeddings.txt", binary=False)

INFO: Access the ontology ...
INFO: There are 3696574 triples in the ontology
INFO: Calculate the ontology projection ...
INFO: Creating ontology graph projection...
INFO: 	Extracting subsumption triples
INFO: 		Time extracting subsumption: 27.165841102600098 seconds 
INFO: 	Extracting equivalence triples
INFO: 		Time extracting equivalences: 23.157968759536743 seconds 
INFO: 	Extracting class membership triples.
INFO: 		Time extracting class membership: 480.71846294403076 seconds 
INFO: 	Extracting sameAs triples
INFO: 		Time extracting sameAs: 0.14655709266662598 seconds 
INFO: 	Extracting triples associated to 116676008
INFO: 		Time extracting triples for property: 6.951526880264282 seconds 
INFO: 	Extracting triples associated to 762705008
INFO: 		Time extracting triples for property: 0.24071192741394043 seconds 
INFO: 	Extracting triples associated to 116686009
INFO: 		Time extracting triples for property: 0.4156510829925537 seconds 
INFO: 	Extracting triples associated to 1181680

KeyboardInterrupt: 